In [1]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
import numpy as np
import gc
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

tqdm.pandas()

gm = gpd.read_parquet('data/gemeenten_concatenated.parquet')

categorisation = pd.read_excel('data/categorisation.xlsx')
L0_cats = categorisation['L0 category'].unique()
L1_cats = categorisation['L1 category'].unique()

In [2]:
gm.head(3)

,gemeentecode,gemeentenaam,H2O,OAD,STED,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,P_00_14_JR,...,RE_L0_1,RE_L1_1,RE_L0_2,RE_L1_2,L0_altieri_0,L1_altieri_0,L0_altieri_1,L1_altieri_1,L0_altieri_2,L1_altieri_2
0,GM0014,Groningen,NEE,3427.0,1.0,1284.0,238147.0,118198.0,119949.0,12.0,...,0.083836,0.092403,0.081681,0.088819,5.404861,6.998096,6.066780,8.816592,6.047942,8.296879
1,GM0034,Almere,NEE,1629.0,2.0,1725.0,222825.0,110589.0,112236.0,18.0,...,0.227224,0.241199,0.215799,0.236978,4.866573,5.913394,5.859722,8.110194,5.807034,7.651038
2,GM0037,Stadskanaal,NEE,840.0,4.0,273.0,32135.0,15844.0,16291.0,14.0,...,0.030424,0.053134,0.033571,0.052019,5.748026,6.916790,4.928690,6.304145,4.768895,5.847494


In [3]:
for idx, row in tqdm(gm.iterrows(), total=gm.shape[0]):
    amenities = gpd.read_parquet(f"data/gm_amenities/amenities_{row['gemeentenaam']}.parquet")
    # amenities = amenities[amenities.geometry.within(row.geometry)]
    for filter_i in [0, 1, 2]:
        
        # get and apply filter
        L0_filter, L1_filter = ec.getfilter(filter_i)
        
        amenities_f = amenities[~amenities.L0_category.isin(L0_filter)]
        if L1_filter:
            for key, value in L1_filter.items():
                amenities_f = amenities_f[
                    ~( (amenities_f.L0_category == key) & (amenities_f.L1_category.isin(value)) )
                ]
        
        for cat in L0_cats:
            gm.loc[idx, f'L1_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L0_category == cat])
        
        for cat in L1_cats:
            gm.loc[idx, f'L0_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L1_category == cat])

    del amenities
    del amenities_f
    gc.collect()

gm.to_parquet('data/gemeenten_concatenated_counted.parquet') 

100%|██████████| 342/342 [03:36<00:00,  1.58it/s]
